In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from nis2pyr.reader import read_nd2file

In [ ]:
import nd2
with nd2.ND2File('/group/jug/ashesh/data/nikola_data/raw/uSplit_14022025_midSNR.nd2') as nd2file:
    data = read_nd2file(nd2file)

In [ ]:
data.shape

In [ ]:
# data = data[0,:,0,...,0]
# ch0 = data[:,0]
# ch1 = data[:,1]
# ch2 = data[:,2]
# ch3 = data[:,3]
# from disentangle.core.tiff_reader import save_tiff
# path = '/group/jug/ashesh/data/nikola_denoising_input/uSplit_14022025_midSNR_channel3.tif'
# save_tiff(path, ch3)

In [ ]:
from disentangle.data_loader.nikola_7D_rawdata_loader import load_one_fpath
data = load_one_fpath('/group/jug/ashesh/data/nikola_data/raw/uSplit_14022025_midSNR.nd2', [0,1,2,3])

## Inspect the bleedthrough

In [ ]:
import matplotlib.pyplot as plt
from disentangle.configs.nikola_7D_config import get_config
from disentangle.data_loader.nikola_7D_rawdata_loader import NikolaChannelList, get_train_val_data
from disentangle.core.data_split_type import DataSplitType

config = get_config()
config.data.enable_gaussian_noise = False
config.data.dset_type = '500ms' # 2ms, 3ms, 5ms, 20ms, 500ms
datadir = '/group/jug/ashesh/data/nikola_data/20240531/'
# datadir = '/group/jug/ashesh/data/nikola_data/raw/'
config.data.channel_idx_list = [
    NikolaChannelList.Ch_A, NikolaChannelList.Ch_B, NikolaChannelList.Ch_C, NikolaChannelList.Ch_D,
    NikolaChannelList.ChBleedthrough_A, NikolaChannelList.ChBleedthrough_B, NikolaChannelList.ChBleedthrough_C, NikolaChannelList.ChBleedthrough_D,

]
data = get_train_val_data(datadir, config.data, DataSplitType.Train,
                            config.training.val_fraction, config.training.test_fraction)

# _,ax = plt.subplots(figsize=(18,6),ncols=3)
# ax[0].imshow(data[0,...,0])
# ax[1].imshow(data[0,...,1])
# ax[2].imshow(data[0,...,2])


In [ ]:
_,ax = plt.subplots(figsize=(18,6),ncols=3)
ch_idx = 0
from matplotlib.colors import LogNorm
norm1 = LogNorm(vmin=10, vmax=data[0,...,ch_idx].max()/2)
norm_bt = LogNorm(vmin=10, vmax=data[0,...,ch_idx+4].max()/2)
print(data[0,...,ch_idx+4].max(), data[0,...,ch_idx].max())
ax[0].imshow(data[0,...,ch_idx], norm=norm1)
ax[1].imshow(data[0,...,ch_idx+4], norm=norm_bt)
ax[2].imshow(data[0,...,1])

In [ ]:
break here

### Creating the denoising data

In [ ]:
from disentangle.configs.nikola_7D_config import get_config
from disentangle.core.data_split_type import DataSplitType
from disentangle.data_loader.nikola_7D_rawdata_loader import get_train_val_data
from disentangle.data_loader.nikola_7D_rawdata_loader import NikolaChannelList


config = get_config()
config.data.dset_type = '5ms'
config.data.enable_gaussian_noise = False
config.data.channel_idx_list = [NikolaChannelList.Ch_A, NikolaChannelList.Ch_B, NikolaChannelList.Ch_C, NikolaChannelList.Ch_D]
datadir = '/group/jug/ashesh/data/nikola_data/20240531/'
data = get_train_val_data(datadir, config.data, DataSplitType.All,
                            config.training.val_fraction, config.training.test_fraction)


In [ ]:
data.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
_,ax = plt.subplots(figsize=(8,4), ncols=2)
img_idx = np.random.randint(0, data.shape[0])
ax[0].imshow(data[img_idx,:1000,:1000,0])
ax[1].imshow(data[img_idx,:1000,:1000,2])

In [ ]:
from disentangle.core.tiff_reader import save_tiff
path_schema = '/group/jug/ashesh/data/nikola_denoising_input/uSplit_{datedir}_{snr}SNR_channel{channel_idx}.tif'
for channel_idx in range(4):
    path = path_schema.format(snr=config.data.dset_type, channel_idx=channel_idx, datedir='20240531')
    save_tiff(path, data[...,channel_idx])
    print('saved to', path)


In [ ]:
from disentangle.core.tiff_reader import  load_tiff
plt.imshow(load_tiff('/group/jug/ashesh/data/nikola_denoising_input/uSplit_20240531_5msSNR_channel0.tif')[0,1000:1200, 600:800])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(data[2,...,2])